# Import necessary libraries

In [ ]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.neighbors import KNeighborsClassifier

from sklearn.linear_model import LinearRegression

import numpy as np
from sklearn.model_selection import KFold

from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

import os
import math
import copy

In [ ]:
# Mouting with google drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Config path and variable

In [ ]:
data_folder = '/content/drive/MyDrive/Modern Problems in Technology'

# Choose one of 2 versions below:
# Uncomment below line to choose scaled data version
# data_version = 'data_processed_withscale.csv'

# Uncomment below line to choose non-scaled data version
data_version = 'data_processed_withscale.csv'


data = pd.read_csv(os.path.join(data_folder, data_version))

data


,lat,lon,NO2,pblh,press,rh,tmp,wspd,dpt_00,dpt_06,dpt_12,dpt_18,pplt_dens,road_dens,cluster
0,21.0491,105.8831,0.000000,0.295189,0.992355,0.503963,0.006589,0.267935,0.180598,0.276074,0.228643,0.194980,0.062616,0.204322,4
1,21.0491,105.8831,0.000000,0.431093,0.970994,0.491827,0.045554,0.192998,0.062569,0.247647,0.354218,0.289970,0.062616,0.204322,4
2,21.0491,105.8831,41.310400,0.241775,0.915679,0.543707,0.081644,0.150712,0.172614,0.324583,0.358503,0.342192,0.062616,0.204322,4
3,21.0491,105.8831,39.379204,0.137111,0.854518,0.683671,0.178665,0.137490,0.286272,0.454138,0.499029,0.491546,0.062616,0.204322,4
4,21.0491,105.8831,39.901879,0.210562,0.830009,0.642043,0.261890,0.161539,0.413801,0.517287,0.572602,0.437929,0.062616,0.204322,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5455,21.0197,105.8147,18.995833,0.346466,0.829110,0.160080,0.227323,0.197763,0.232337,0.278541,0.204074,0.253429,1.000000,0.793592,4
5456,21.0197,105.8147,23.241667,0.301871,0.758730,0.249164,0.333443,0.112616,0.445943,0.492757,0.243412,0.343498,1.000000,0.793592,4
5457,21.0197,105.8147,16.816667,0.241945,0.703640,0.501023,0.452160,0.047686,0.597599,0.629433,0.505862,0.644568,1.000000,0.793592,1
5458,21.0197,105.8147,21.595833,0.357267,0.767949,0.479585,0.471869,0.227892,0.560463,0.559629,0.590930,0.643036,1.000000,0.793592,1


## Reading data version

In [ ]:
data = pd.read_csv(os.path.join(data_folder, data_version))

data


,lat,lon,NO2,pblh,press,rh,tmp,wspd,dpt_00,dpt_06,dpt_12,dpt_18,pplt_dens,road_dens,cluster
0,21.0491,105.8831,0.000000,0.295189,0.992355,0.503963,0.006589,0.267935,0.180598,0.276074,0.228643,0.194980,0.062616,0.204322,4
1,21.0491,105.8831,0.000000,0.431093,0.970994,0.491827,0.045554,0.192998,0.062569,0.247647,0.354218,0.289970,0.062616,0.204322,4
2,21.0491,105.8831,41.310400,0.241775,0.915679,0.543707,0.081644,0.150712,0.172614,0.324583,0.358503,0.342192,0.062616,0.204322,4
3,21.0491,105.8831,39.379204,0.137111,0.854518,0.683671,0.178665,0.137490,0.286272,0.454138,0.499029,0.491546,0.062616,0.204322,4
4,21.0491,105.8831,39.901879,0.210562,0.830009,0.642043,0.261890,0.161539,0.413801,0.517287,0.572602,0.437929,0.062616,0.204322,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5455,21.0197,105.8147,18.995833,0.346466,0.829110,0.160080,0.227323,0.197763,0.232337,0.278541,0.204074,0.253429,1.000000,0.793592,4
5456,21.0197,105.8147,23.241667,0.301871,0.758730,0.249164,0.333443,0.112616,0.445943,0.492757,0.243412,0.343498,1.000000,0.793592,4
5457,21.0197,105.8147,16.816667,0.241945,0.703640,0.501023,0.452160,0.047686,0.597599,0.629433,0.505862,0.644568,1.000000,0.793592,1
5458,21.0197,105.8147,21.595833,0.357267,0.767949,0.479585,0.471869,0.227892,0.560463,0.559629,0.590930,0.643036,1.000000,0.793592,1


In [ ]:
# data = data[data.cluster != 2]
# Unique cluster value
data['cluster'].unique()

array([4, 3, 1, 0, 2])

## Remove Invalid NO2 values

In [ ]:
# Run this if you want to remove invalid no2 values

data = data[(data.NO2 > 0) & (data.NO2 < 100)]
data[data.NO2 > 100]

,lat,lon,NO2,pblh,press,rh,tmp,wspd,dpt_00,dpt_06,dpt_12,dpt_18,pplt_dens,road_dens,cluster


## Drop some columns of data



In [ ]:
def drop_cols(df, colName):
    if colName in df:
      df.drop(colName, axis='columns', inplace=True)

# drop_cols(data, "press")

## Create multiple datatset by dropping each colums of original dataset


In [ ]:
columns = data.columns
# for col in range(3, 15):
#   print(columns[col])
dropDatas = [data]
# print(dropDatas[0])
for i in range(3, 15):
  print(columns[i])
  data_copy = copy.deepcopy(data)
  drop_cols(data_copy, columns[i])
  dropDatas.append(data_copy)
  # print(len(dropDatas[i-3].columns))
len(dropDatas)

pblh
press
rh
tmp
wspd
dpt_00
dpt_06
dpt_12
dpt_18
pplt_dens
road_dens
cluster


13

In [ ]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100
len(columns)

15

## Convert to numpy for feeding model


In [ ]:
for idx in range(2, 15):
  print("---------------------------------------------")
  print("I: " + str(idx))
  tmpData = dropDatas[idx-2]
  X = tmpData.drop('NO2', axis = 'columns')
  X_arr = X.to_numpy()

  y = tmpData['NO2']
  y_arr = y.to_numpy()

  print(X.shape, y.shape)


  # List of RMSE on all folds
  rmses_rf = []
  mapes_rf = []
  r2s_rf = []

  rmses_lr = []
  mapes_lr = []
  r2s_lr = []

  # Fold tracking
  i = 1
  # Depth of Random Forest
  depth = 50
  # Number of split
  split_number = 5

  kf = KFold(n_splits=split_number, random_state=None, shuffle=True)
  kf.get_n_splits(X)

  for train_index, test_index in kf.split(X):
    min_rmse_rf = 10000
    min_r2_rf = 10000
    min_mape_rf = 10000

    min_rmse_lr = 10000
    min_r2_lr = 10000
    min_mape_lr = 10000

    # print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X_arr[train_index], X_arr[test_index]
    y_train, y_test = y_arr[train_index], y_arr[test_index]
    print(len(train_index), len(test_index))

    print('Fold {}:'.format(i))

    # Random Forest
    for d in range(1, depth):
      rf = RandomForestRegressor(n_estimators = d, random_state = 20)
      rf.fit(X_train, y_train)
      rf_predicts = rf.predict(X_test)
      # Calculate RMSE
      rmse_rf = math.sqrt(mean_squared_error(y_test, rf_predicts))
      # Calculate R2 Score
      r2_rf = r2_score(y_test, rf_predicts)
      # Calculate mean absolute error (MAE NOT MAPE)
      # mape_rf = mean_absolute_error(y_test, rf_predicts)
      mape_rf = mean_absolute_percentage_error(y_test, rf_predicts)
      
      # Saving best errors
      if rmse_rf < min_rmse_rf:
        min_rmse_rf = rmse_rf
      if r2_rf < min_r2_rf:
        min_r2_rf = r2_rf
      if mape_rf < min_mape_rf:
        min_mape_rf = mape_rf

    rmses_rf.append(min_rmse_rf)
    r2s_rf.append(min_r2_rf)
    mapes_rf.append(min_mape_rf)

    print('Min RMSE for Random Forest:'.format(i), min_rmse_rf)
    print('Min R2 for Random Forest:'.format(i), min_r2_rf)
    print('Min MAPE for Random Forest:'.format(i), min_mape_rf)
    print()

    # Linear Regression
    lr = LinearRegression()
    lr.fit(X_train, y_train)
    lr_predicts = lr.predict(X_test)
    # Calculate RMSE
    rmse_lr = math.sqrt(mean_squared_error(y_test, lr_predicts))
    # Calculate R2 Score
    r2_lr = r2_score(y_test, lr_predicts)
    # Calculate mean absolute error (MAE NOT MAPE)
    # mape_lr = mean_absolute_error(y_test, lr_predicts)
    mape_lr = mean_absolute_percentage_error(y_test, rf_predicts)

    if rmse_lr < min_rmse_lr:
      min_rmse_lr = rmse_lr
    if r2_lr < min_r2_lr:
      min_r2_lr = r2_lr
    if mape_lr < min_mape_lr:
      min_mape_lr = mape_lr

    # a={'true': y_test, 'predict':predicts}
    # display(pd.DataFrame(a))

    rmses_lr.append(min_rmse_lr)
    r2s_lr.append(min_r2_lr)
    mapes_lr.append(min_mape_lr)

    print('Min RMSE for Linear Regression:'.format(i), min_rmse_lr)
    print('Min R2 for Linear Regression:'.format(i), min_r2_lr)
    print('Min MAPE for Linear Regression:'.format(i), min_mape_lr)
    print()
    print()

    i += 1
  if (idx == 2):
      print("Original data")
      print()
  else:
      print("Data without " + columns[idx])
      print()
  print()
  print('Average RMSE on all folds using Random Forest:', np.average(rmses_rf))
  print('Average R2 on all folds using Random Forest:', np.average(r2s_rf))
  print('Average MAPE on all folds using Random Forest:', np.average(mapes_rf))

  print()
  print('Average RMSE on all folds using Linear Regression:', np.average(rmses_lr))
  print('Average R2 on all folds using Linear Regression:', np.average(r2s_lr))
  print('Average MAPE on all folds using Linear Regression:', np.average(mapes_lr))


---------------------------------------------
I: 2
(4679, 14) (4679,)
3743 936
Fold 1:
Min RMSE for Random Forest: 9.991604866911246
Min R2 for Random Forest: 0.1489616178816443
Min MAPE for Random Forest: 36.59173906842607

Min RMSE for Linear Regression: 13.84938686261356
Min R2 for Linear Regression: 0.12694230556182384
Min MAPE for Linear Regression: 36.60675456622807


3743 936
Fold 2:
Min RMSE for Random Forest: 9.8128567040693
Min R2 for Random Forest: 0.1884066117753448
Min MAPE for Random Forest: 42.39700522461175

Min RMSE for Linear Regression: 13.868755334729949
Min R2 for Linear Regression: 0.15895055061724173
Min MAPE for Linear Regression: 42.687118858214056


3743 936
Fold 3:
Min RMSE for Random Forest: 10.103970646087406
Min R2 for Random Forest: 0.14929667673031344
Min MAPE for Random Forest: 33.46783444396152

Min RMSE for Linear Regression: 14.244384759273736
Min R2 for Linear Regression: 0.13752686938473557
Min MAPE for Linear Regression: 33.50154888291777


3743 9